# Linear Models

In [55]:
import pandas as pd
import numpy as np

In [58]:
clean = pd.read_csv("../data/clean.csv")
clean.head()

,crop_count,crop_name_1,crop_harvest_1,crop_intercrop_1,ID_PROJ,ID_COUNTRY,YEAR,SURVEY_ID,ID_HH,Country,...,value_livestock_production_USD_PPP_pHH_Yr,value_livestock_prod_consumed_USD_PPP_pHH_Yr,Market_Orientation,Livestock_Orientation,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,NrofMonthsWildFoodCons,Gender_MaleControl,Gender_FemaleControl,continent
0,NaN,groundnut,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_1_1,Tanzania,...,38.500640,0.000000,1.000000,0.049505,3650.565755,0.000000,0,0.000000,1.000000,africa
1,NaN,millet,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_2_1,Tanzania,...,0.000000,0.000000,1.000000,0.000000,3632.448997,0.000000,4,0.499999,0.500001,africa
2,NaN,groundnut,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_3_1,Tanzania,...,128.335467,0.000000,1.000000,0.129870,5314.779021,1750.000000,1,1.000000,0.000000,africa
3,NaN,cowpea,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_4_1,Tanzania,...,0.000000,0.000000,NaN,NaN,257.234727,257.234727,0,NaN,NaN,africa
4,NaN,sesame,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_5_1,Tanzania,...,233.994961,23.102694,0.931377,0.695041,925.310354,22.070240,0,1.000000,0.000000,africa


In [59]:
from Functions import Cleaning_Functions
fun = Cleaning_Functions()

In [60]:
#clean.loc[:, "Market_Orientation"]
clean.head()

,crop_count,crop_name_1,crop_harvest_1,crop_intercrop_1,ID_PROJ,ID_COUNTRY,YEAR,SURVEY_ID,ID_HH,Country,...,value_livestock_production_USD_PPP_pHH_Yr,value_livestock_prod_consumed_USD_PPP_pHH_Yr,Market_Orientation,Livestock_Orientation,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,NrofMonthsWildFoodCons,Gender_MaleControl,Gender_FemaleControl,continent
0,NaN,groundnut,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_1_1,Tanzania,...,38.500640,0.000000,1.000000,0.049505,3650.565755,0.000000,0,0.000000,1.000000,africa
1,NaN,millet,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_2_1,Tanzania,...,0.000000,0.000000,1.000000,0.000000,3632.448997,0.000000,4,0.499999,0.500001,africa
2,NaN,groundnut,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_3_1,Tanzania,...,128.335467,0.000000,1.000000,0.129870,5314.779021,1750.000000,1,1.000000,0.000000,africa
3,NaN,cowpea,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_4_1,Tanzania,...,0.000000,0.000000,NaN,NaN,257.234727,257.234727,0,NaN,NaN,africa
4,NaN,sesame,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_5_1,Tanzania,...,233.994961,23.102694,0.931377,0.695041,925.310354,22.070240,0,1.000000,0.000000,africa


In [65]:
#for market orientash
market_data = fun.delete_id_columns(clean) #1
print("1")
market_data, pred_market = fun.drop_response_rows_with_NAs(market_data, "Market_Orientation") #2
print("2")
market_data = fun.replace_NAN_with_na(market_data) #3
print("3")
market_data = fun.entry_to_lowercase(market_data) #4
print("4")
market_data = fun.remove_underscores_spaces(market_data) #5
print("5")
market_data = fun.convert_to_categorical(market_data) #6
print("6")
market_data = fun.impute_data(market_data)
print("7")

1
2
3
4
5
6
7


In [66]:
clean.loc[:, "Market_Orientation"]

0        1.000000
1        1.000000
2        1.000000
3             NaN
4        0.931377
           ...   
13305    1.000000
13306         NaN
13307    0.456522
13308         NaN
13309    1.000000
Name: Market_Orientation, Length: 13310, dtype: float64

In [78]:
#X and Y for market orientation
from sklearn.model_selection import train_test_split

X = market_data.loc[:, market_data.columns != 'Market_Orientation']
y = market_data.loc[:, "Market_Orientation"]
(Xtrain, Xtest, ytrain, ytest) = train_test_split(X,y,test_size = .3)

Xtrain, Xtest = fun.impute_data(Xtrain, Xtest)

TypeError: impute_data() takes 2 positional arguments but 3 were given

In [ ]:
#for PPI Threshold
market_PPI = fun.delete_id_columns(clean) #1
print("1")
market_PPI, pred_PPI = fun.drop_response_rows_with_NAs(market_PPI, "PPI_Threshold") #2
print("2")
market_PPI = fun.replace_NAN_with_na(market_PPI) #3
print("3")
market_PPI = fun.entry_to_lowercase(market_PPI) #4
print("4")
market_PPI = fun.remove_underscores_spaces(market_PPI) #5
print("5")
market_PPI = fun.convert_to_categorical(market_PPI) #6
print("6")
market_PPI = fun.impute_data(market_PPI)
print("7")

In [ ]:
#X and Y for PPI_threshold
from sklearn.model_selection import train_test_split

X_p = market_data.loc[:, market_data.columns != 'PPI_Threshold']
y_p = market_data.loc[:, "PPI_Threshold"]
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_m,y_m,test_size = .3)

X_train_p, X_test_p = fun.impute_data(X_train_p, X_test_p)

X_train_p